**Projeto Análise de Dados**


*Este bloco de notas contém todo o código usado para seleção, limpeza e transformação (extração, transformação, carregamento ou ETL, do inglês Extract Transform Load)  das fontes de dados de bairros e atividades econômicas para aplicação das técnicas de análise de dados a serem estudadas*

Este bloco de notas recebe como entrada um arquivo de atividades econômicas do Estado do Espírito Santo (ATIVECO_ES) anonimizado e dois arquivos públicos extraídos das plataformas do IBGE. Um arquivo que descreve a estrutura detalhada da CNAE-Subclasses 2.3 (códigos nacionais e suas respectivas atividades econômicas). O outro arquivo contém os códigos e os bairros dos municípios da Grande Vitória (GV): Cariacica, Serra, Viana, Vila Velha e Vitória. Os códigos dos bairros da GV foram definidos pelo IBGE pois seus municípios tiveram suas leis de bairros aprovadas até agosto de 2010.


Como saída do processamento desse bloco de notas, tem-se um arquivo contendo uma tabela cujas linhas contêm os bairros presentes no arquivo ATIVECO_ES (mapeadas para os códigos de bairros do IBGE) e cujas colunas contêm as atividades presentes no arquivo ATIVECO_ES (mapeadas para os códigos de atividades do CNAE). 
O conteúdo de cada célula da tabela contém a quantidade daquela atividade (coluna) naquele bairro (linha).
O arquivo de saida servirá de entrada para o próximo módulo (bloco de notas) de Exploração dos Dados.

# Setup

Primeiro, vamos importar alguns módulos comuns, garantir que o MatplotLib plote figuras em linha e preparar uma função para salvar as figuras. Também verificamos se o Python 3.5 ou posterior está instalado (embora o Python 2.x possa funcionar, ele está obsoleto, por isso recomendamos fortemente que seja usado o Python 3), bem como o Scikit-Learn ≥0.20.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

DATA_PATH_INPUT= os.path.join(".", "INPUT")
DATA_PATH_OUTPUT = os.path.join(".","OUTPUT")

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(DATA_PATH_OUTPUT_BA, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Carregando os dados dos arquivos físicos

In [2]:
import os
import tarfile


def fetch_data_ativeco_ES(data_path=DATA_PATH_INPUT):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    tgz_path = os.path.join(data_path, "AtividadesEconomicas_ES.csv.tar.gz")
    data_tgz = tarfile.open(tgz_path)
    data_tgz.extractall(path=data_path)
    data_tgz.close()

    

In [3]:
import pandas as pd

def load_data_ativeco_ES(data_path=DATA_PATH_INPUT):
    ### The csv was extracted from a tar.gz by function fetch_data() ### main choise
    csv_path = os.path.join(data_path, "AtividadesEconomicas_ES.csv")
    return pd.read_csv(csv_path)
    
    ### if we want load from the original xlsx file ### slower than main choise
    #xlsx_path = os.path.join(data_path, "ativeco_ES_2021_Ver01.xlsx")
    #wb = pd.read_excel(xlsx_path)
    #return wb     
    ### if we want generate a csv from the original xlsx file ### 
    #df = pd.DataFrame(wb)
    #csv_path = os.path.join(data_path, "ativeco_ES_2021_Ver01.csv")
    #df.to_csv(csv_path,sep=',',index=False, encoding='utf-8')
   


# Limpeza e preparação da base de dados ativeco_ES


## Apenas municípios da Grande Vitória

 - CARIACICA
 - SERRA
 - VIANA
 - VILA VELHA
 - VITORIA

In [4]:
### Load original ativeco_ES database
fetch_data_ativeco_ES()
ativeco_ES = load_data_ativeco_ES()
#print(ativeco_ES.shape)

### turn all columns into object
ativeco_ES["CNPJ"] = ativeco_ES["CNPJ"].astype(str)
ativeco_ES["NR_CEP"] = ativeco_ES["NR_CEP"].astype(str)

### Select only municipalities from grande vitoria
ativeco_ESGV = ativeco_ES.loc[ativeco_ES['NO_MUNICIPIO'].isin(["CARIACICA","SERRA","VIANA","VILA VELHA","VITÓRIA"])]

### Reindex rows starting again from zero
# If we do not want to add the new column, we can use the drop parameter
# We can use the parameter inplace to reset the index in the existing DataFrame rather than create a new copy
ativeco_ESGV.reset_index(inplace=True,drop=True)

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "00ativeco_ESGV.csv")
ativeco_ESGV.to_csv(csv_path, index=True, header=True)
print("00", ativeco_ESGV.shape)

00 (278060, 11)



## Limpeza CNPJ 

- Remover linhas com NaN na coluna CNPJ
- Verificar ocorrência de mais de um CNPJ e remover


###  Remover linhas com NaN na coluna CNPJ

In [5]:
### number of rows
print("00", ativeco_ESGV.shape)

### Remove NaNs (which currently contain the string 'nan' due to int64 to object conversion)
ativeco_ESGV = ativeco_ESGV.replace('nan',np.nan)
ativeco_ESGV = ativeco_ESGV.dropna(subset=["CNPJ"])

### Reindex rows starting again from zero
# If we do not want to add the new column, we can use the drop parameter.
# We can use the parameter inplace to reset the index in the existing DataFrame rather than create a new copy.
ativeco_ESGV.reset_index(inplace=True,drop=True)


#### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "01ativeco_ESGV_CNPJ-Dropna.csv")
ativeco_ESGV.to_csv(csv_path, index=True, header=True)
print("01", ativeco_ESGV.shape)

00 (278060, 11)
01 (269243, 11)


### Verificar ocorrência de mais de um CNPJ e remover

In [6]:
### Prints how many times a CNPJ appears
print(ativeco_ESGV["CNPJ"].value_counts())

12100748000145.0    3
7188015000174.0     2
22144512000164.0    2
27418289000110.0    2
14203274000192.0    2
                   ..
4604275000195.0     1
4601143000100.0     1
4642928000120.0     1
4650498000199.0     1
21723386000130.0    1
Name: CNPJ, Length: 269127, dtype: int64


In [7]:
### Print example of a CNPJ that appears more than once
# Checking the JUCCES companies consultation (https://jucees.es.gov.br/consulta-empresas)
# in the case of the example GIOVANNY is the owner of POLO BEER COMERCIO and entries are equivalent
# same happened with other examples, remove duplicate and keep either one
print(ativeco_ESGV[ativeco_ESGV["CNPJ"] == "26568240000180.0"][["NOMEEMPRESA","CNPJ","DS_LOGRADOURO","NO_BAIRRO","DS_ATIVIDADE"]])

                              NOMEEMPRESA              CNPJ DS_LOGRADOURO  \
159844         POLO BEER COMERCIO LTDA ME  26568240000180.0         CEDRO   
177210  GIOVANNY SOARES SILVA 15210860795  26568240000180.0    RUA BRAUNA   

                NO_BAIRRO                   DS_ATIVIDADE  
159844  VISTA DA SERRA II  COMÉRCIO VAREJISTA DE BEBIDAS  
177210  VISTA DA SERRA II  COMÉRCIO VAREJISTA DE BEBIDAS  


In [8]:
### keep only the first occurrence of the line that has a duplicate CNPJ
print("01", ativeco_ESGV.shape)
ativeco_ESGV = ativeco_ESGV.drop_duplicates(subset='CNPJ', keep='first')

### Reindex rows starting again from zero
# If we do not want to add the new column, we can use the drop parameter.
# We can use the parameter inplace to reset the index in the existing DataFrame rather than create a new copy.
ativeco_ESGV.reset_index(inplace=True,drop=True)


#### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "02ativeco_ESGV_CNPJ-DropDuplicates.csv")
ativeco_ESGV.to_csv(csv_path, index=True, header=True)
print("02", ativeco_ESGV.shape)

01 (269243, 11)
02 (269127, 11)



## Limpeza BAIRRO 

### Remover linhas com NaN na coluna NO_BAIRRO
- Total linhas           211927
- Coluna NO_BAIRRO       211920 non-null


In [9]:
### Remove NaNs
print("02", ativeco_ESGV.shape)
ativeco_ESGV = ativeco_ESGV.dropna(subset=["NO_BAIRRO"])

### Reindex rows starting again from zero
# If we do not want to add the new column, we can use the drop parameter.
# We can use the parameter inplace to reset the index in the existing DataFrame rather than create a new copy.
ativeco_ESGV.reset_index(inplace=True,drop=True)

#### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "03ativeco_ESGV_Bairro-Dropna.csv")
ativeco_ESGV.to_csv(csv_path, index=True, header=True)
print("03", ativeco_ESGV.shape)

02 (269127, 11)
03 (269113, 11)



## Substituir todos os NaNs 

- Total linhas           211920
- Coluna MEI             181975 non-null -> substituir NaN por 'N'
- Coluna DS_LOGRADOURO   211919 non-null -> substituir NaN por ''
- Coluna DS_NUMERO       211729 non-null -> substituir NaN por 'S/N'
- Coluna DS_COMPLEMENTO   91242 non-null -> substituir NaN por ''
- Coluna NR_CEP          211862 non-null -> substituir NaN por '29000000'


In [10]:
ativeco_ESGV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269113 entries, 0 to 269112
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   NOMEEMPRESA     269113 non-null  object
 1   PORTE           269113 non-null  object
 2   MEI             226000 non-null  object
 3   CNPJ            269113 non-null  object
 4   DS_LOGRADOURO   269112 non-null  object
 5   DS_NUMERO       268907 non-null  object
 6   DS_COMPLEMENTO  126925 non-null  object
 7   NO_BAIRRO       269113 non-null  object
 8   NR_CEP          269029 non-null  object
 9   NO_MUNICIPIO    269113 non-null  object
 10  DS_ATIVIDADE    269113 non-null  object
dtypes: object(11)
memory usage: 22.6+ MB


In [11]:
print("03", ativeco_ESGV.shape)
ativeco_ESGV["MEI"]            = ativeco_ESGV["MEI"].replace(np.nan, 'N')
ativeco_ESGV["DS_LOGRADOURO"]  = ativeco_ESGV["DS_LOGRADOURO"].replace(np.nan, '')
ativeco_ESGV["DS_NUMERO"]      = ativeco_ESGV["DS_NUMERO"].replace(np.nan, 'S/N')
ativeco_ESGV["DS_COMPLEMENTO"] = ativeco_ESGV["DS_COMPLEMENTO"].replace(np.nan, '')
ativeco_ESGV["NR_CEP"]         = ativeco_ESGV["NR_CEP"].replace(np.nan, '290000000')

#### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "04ativeco_ESGV-ReplaceNaN.csv")
ativeco_ESGV.to_csv(csv_path, index=True, header=True)
print("04", ativeco_ESGV.shape)

ativeco_ESGV.info()

03 (269113, 11)
04 (269113, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269113 entries, 0 to 269112
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   NOMEEMPRESA     269113 non-null  object
 1   PORTE           269113 non-null  object
 2   MEI             269113 non-null  object
 3   CNPJ            269113 non-null  object
 4   DS_LOGRADOURO   269113 non-null  object
 5   DS_NUMERO       269113 non-null  object
 6   DS_COMPLEMENTO  269113 non-null  object
 7   NO_BAIRRO       269113 non-null  object
 8   NR_CEP          269113 non-null  object
 9   NO_MUNICIPIO    269113 non-null  object
 10  DS_ATIVIDADE    269113 non-null  object
dtypes: object(11)
memory usage: 22.6+ MB



## Padronização strings

- Todas maiúsculas
- remover acentuações e caracteres especiais
- criar colunas chaves para merge com IBGE: CH_ATIVIDADE, CH_MUNICIPIO_BAIRRO



###  Criar colunas chaves para merge com IBGE: CH_ATIVIDADE


In [12]:
def correct_names(name):
    name = name.replace('Á', 'A').replace('À', 'A').replace('Â', 'A').replace('Ã', 'A')
    name = name.replace('É', 'E').replace('È', 'E').replace('Ê', 'E')
    name = name.replace('Í', 'I').replace('Ì', 'I').replace('Î', 'I')
    name = name.replace('Ó', 'O').replace('Ò', 'O').replace('Ô', 'O').replace('Õ', 'O')
    name = name.replace('Ú', 'U').replace('Ù', 'U').replace('Û', 'U').replace('Ũ', 'U').replace('Ü', 'U')
    name = name.replace('Ç', 'C')
    #removing all spaces at the beginning, end and between of a string
    name = name.replace(" ","")
    #removing linebreak and tabs
    name = name.replace("\n","")
    name = name.replace("\t","")
    return name


print("04", ativeco_ESGV.shape)
### converting whole dataframe from lower case to upper case
ativeco_ESGV = ativeco_ESGV.apply(lambda x: x.astype(str).str.upper())

### creating a key column and removing accentuation, spaces, linebreak and tabs from it
# the key column will be used for the merge levenstein diference with CNAE atividades
ativeco_ESGV["CH_ATIVIDADE"] = ativeco_ESGV["DS_ATIVIDADE"].apply(correct_names)

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "05ativeco_ESGV-StringStandart_CH_ATIVIDADE.csv")
ativeco_ESGV.to_csv(csv_path, index=True, header=True)
print("05", ativeco_ESGV.shape)


04 (269113, 11)
05 (269113, 12)



###  Criar colunas chaves para merge com IBGE: CH_ATIVIDADE, CH_MUNICIPIO_BAIRRO


In [13]:
print("05", ativeco_ESGV.shape)
### creating a key column and removing accentuation, spaces, linebreak and tabs from it
# the key column will be used for the merge levenstein diference with IBGE município+bairro
ativeco_ESGV["CH_MUNICIPIO_BAIRRO"] = ativeco_ESGV['NO_MUNICIPIO']+ativeco_ESGV["NO_BAIRRO"]
ativeco_ESGV["CH_MUNICIPIO_BAIRRO"] = ativeco_ESGV["CH_MUNICIPIO_BAIRRO"].apply(correct_names)

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "06ativeco_ESGV-StringStandart_CH_ATIVIDADE_CH_MUNICIPIO_BAIRRO.csv")
ativeco_ESGV.to_csv(csv_path, index=True, header=True)
print("06", ativeco_ESGV.shape)

05 (269113, 12)
06 (269113, 13)


# Limpeza e preparação da base de dados IBGE-Atividades (CNAE23)


## Remoção colunas não relevantes

- Nome da atividade como coluna "Denominacao"
- Nome da Divisão como "DivNome"
- Manter apenas colunas "Divisão","Subclasse","Denominacao" sem NaN

In [15]:
import pandas as pd

def load_data_IBGE(data_path=DATA_PATH_INPUT):
    ### The csv was extracted from a tar.gz by function fetch_data() ### main choise
    xlsx_path = os.path.join(data_path, "AtividadesEconomicas_IBGE.xlsx")
    return pd.read_excel(xlsx_path, header=3,dtype=object)

In [16]:
### Original base in multindex format
CNAE23 = load_data_IBGE()

### name of Atividade as column "Denominacao"
CNAE23.columns.values[5] = "Denominacao"

### turning multindex format into flat (traditional) format
mask_frame = CNAE23.iloc[:,4::-1].isna().cumprod(axis=1).iloc[:,4::-1].astype(bool)
mask_frame['Denominacao']=False
CNAE23 = CNAE23[:].ffill().mask(mask_frame)
CNAE23 = CNAE23.replace(np.nan,'')


#### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "100CNAE_Flat.csv")
CNAE23.to_csv(csv_path, index=True, header=True)


## Criação da coluna "DivNome"
- Enquanto a coluna "Divisão" possui o código numério da divisão (agrupamento de atividades relacionadas), a coluna "DivNome" conterá o nome da divisão (agrupamento de atividades relacionadas)

In [17]:
### Assignment by reference, keeping only the columns of interest
# Column "Grupo" will be transformed in "DivNome"
CNAE23 = CNAE23[["Divisão","Grupo","Subclasse","Denominacao"]]

### ensuring that any void will be NaN
CNAE23 = CNAE23.replace('',np.nan)

### filling in the "subclass" blanks, more detailed (lower) code that uniquely identifies an activity
# empty spaces occur in rows with the definition of,e.g., "class" codes(already dropped),
# due to original multiindex format
CNAE23['Subclasse'] = CNAE23['Subclasse'].fillna(method='bfill')

### grouping activities by "division" and creating the column 'DivName' with the name of the division
### removing codes higher than Divisão
CNAE23 = CNAE23.dropna(subset=['Divisão'])
### creating to "DivNome" and assigning the name to its respective division read from "Denominacao"
CNAE23['Grupo'] = CNAE23['Grupo'].fillna(CNAE23['Denominacao'])
CNAE23 = CNAE23.rename(columns={'Grupo': 'DivNome'})


# Loops through all columns of the dataset propagating division name, change name when division changes
CNAE23.reset_index(inplace=True,drop=True)
for i, infos in CNAE23.iterrows():
    # Defining the first division
    if i == 0:
        Divisao_atual = infos.Divisão
        DivNome_atual = infos.DivNome
    else:
        # Checks if the divisão has been changed, and when it changes, updates the DivNome for the new divisão
        if infos.Divisão != Divisao_atual:
            #print(f'Houve mudança de divisão {infos.Divisão} de nome {infos.DivNome}')
            Divisao_atual = infos.Divisão
            DivNome_atual = infos.DivNome
        else:
            infos.DivNome = DivNome_atual


### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "101CNAE_DivNome.csv")
CNAE23.to_csv(csv_path, index=True, header=True)
CNAE23.head()

,Divisão,DivNome,Subclasse,Denominacao
0,01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",0111-3/01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"
1,01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",0111-3/01,Produção de lavouras temporárias
2,01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",0111-3/01,Cultivo de cereais
3,01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",0111-3/01,Cultivo de arroz
4,01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS",0111-3/02,Cultivo de milho



## Padronização strings

- Todas maiúsculas
- remover acentuações e caracteres especiais
- coluna "Denominacao" como coluna chave (nome original da atividade foi mantida na base ativeco_ES)

In [18]:
def correct_names(name):
    name = name.replace('Á', 'A').replace('À', 'A').replace('Â', 'A').replace('Ã', 'A')
    name = name.replace('É', 'E').replace('È', 'E').replace('Ê', 'E')
    name = name.replace('Í', 'I').replace('Ì', 'I').replace('Î', 'I')
    name = name.replace('Ó', 'O').replace('Ò', 'O').replace('Ô', 'O').replace('Õ', 'O')
    name = name.replace('Ú', 'U').replace('Ù', 'U').replace('Û', 'U').replace('Ũ', 'U').replace('Ü', 'U')
    name = name.replace('Ç', 'C')
    #removing all spaces at the beginning, end and between of a string
    name = name.replace(" ","")
    #removing linebreak and tabs
    name = name.replace("\n","")
    name = name.replace("\t","")
    return name

#converting whole dataframe from lower case to upper case
CNAE23 = CNAE23.apply(lambda x: x.astype(str).str.upper())

### creating a key column and removing accentuation, spaces, linebreak and tabs from it
# the key column will be used for the merge levenstein diference with ativeco_ES
CNAE23["CH_ATIVIDADE"] = CNAE23["Denominacao"]
CNAE23["CH_ATIVIDADE"] = CNAE23["CH_ATIVIDADE"].apply(correct_names)

#### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "102CNAE-StringStandart.csv")
CNAE23.to_csv(csv_path, index=True, header=True)


# Limpeza e preparação da base de dados IBGE-Bairros


## Remoção colunas não relevantes

- Remover coluna "Estado"
- Re-ordenar colunas para "Cód.","Município","Bairro"
- Não houve preocupação com NaN pois eles não existem nessa basse

In [22]:
import pandas as pd

def load_data_IBGE_bairros(data_path=DATA_PATH_INPUT):
    #xlsx_path = os.path.join(data_path, "Tabela_Bairros_GV.xlsx")
    #return pd.read_excel(xlsx_path, header=3,dtype=object)
    csv_path = os.path.join(data_path, "BairrosGV_IBGE.csv")
    return pd.read_csv(csv_path)

In [23]:
IBGE_Bairros = load_data_IBGE_bairros()

### Remove "Estado" column and reorder columns
columns_titles = ["Cód.","Município","Bairro"]
IBGE_Bairros=IBGE_Bairros.reindex(columns=columns_titles)

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "200IBGE_Bairros_Relevantes.csv")
IBGE_Bairros.to_csv(csv_path, index=True, header=True)


## Padronização strings

- Todas maiúsculas
- remover acentuações e caracteres especiais
- criar coluna chave para merge com ativeco_ES: CH_MUNICIPIO_BAIRRO

In [24]:
def correct_names(name):
    name = name.replace('Á', 'A').replace('À', 'A').replace('Â', 'A').replace('Ã', 'A')
    name = name.replace('É', 'E').replace('È', 'E').replace('Ê', 'E')
    name = name.replace('Í', 'I').replace('Ì', 'I').replace('Î', 'I')
    name = name.replace('Ó', 'O').replace('Ò', 'O').replace('Ô', 'O').replace('Õ', 'O')
    name = name.replace('Ú', 'U').replace('Ù', 'U').replace('Û', 'U').replace('Ũ', 'U').replace('Ü', 'U')
    name = name.replace('Ç', 'C')
    #removing all spaces at the beginning, end and between of a string
    name = name.replace(" ","")
    #removing linebreak and tabs
    name = name.replace("\n","")
    name = name.replace("\t","")
    return name


#converting whole dataframe from lower case to upper case
IBGE_Bairros = IBGE_Bairros.apply(lambda x: x.astype(str).str.upper())

### creating a key column and removing accentuation, spaces, linebreak and tabs from it
# the key column will be used for the merge levenstein diference with ativeco_ES
IBGE_Bairros["CH_MUNICIPIO_BAIRRO"] = IBGE_Bairros['Município']+IBGE_Bairros["Bairro"]
IBGE_Bairros["CH_MUNICIPIO_BAIRRO"] = IBGE_Bairros["CH_MUNICIPIO_BAIRRO"].apply(correct_names)

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "201IBGE_Bairros_StringStandart.csv")
IBGE_Bairros.to_csv(csv_path, index=True, header=True)


# MAPEAMENTO BAIRRO

### Cria tabela (de ativeco_ES) com agrupamentos de localidade (MUNICIPIOBAIRRO)
- usar coluna CH_MUNICIPIO_BAIRRO
- tabela [CH_MUNICIPIO_BAIRRO,DS_ATIVIDADE,count]
    - DS_ATIVIDADE = quantidade de atividades diferentes na localidade
    - count = quantidade de atividades na localidade

In [25]:
### Assignment by copy and not by reference, keeping only the columns of interest
Ativ_Bairro = ativeco_ESGV.loc[:,["NO_MUNICIPIO","NO_BAIRRO","DS_ATIVIDADE","CH_ATIVIDADE","CH_MUNICIPIO_BAIRRO"]]

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "07ativeco_ESGV-Columns.csv") #Somatorio_atividades_aposmerge.csv
Ativ_Bairro.to_csv(csv_path, index=True, header=True)
print("07", Ativ_Bairro.shape)


### Number of different activities per bairro
Group_Ativ_Bairro_SUM = Ativ_Bairro.groupby(["CH_MUNICIPIO_BAIRRO"],as_index=False)["DS_ATIVIDADE"].nunique()
Group_Ativ_Bairro_SUM = Group_Ativ_Bairro_SUM.sort_values(["CH_MUNICIPIO_BAIRRO"])

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "08ativeco_ESGV-Columns-Atividade_por_Bairro.csv") #Somatorio_atividades_aposmerge.csv
Group_Ativ_Bairro_SUM.to_csv(csv_path, index=True, header=True)
print("08", Group_Ativ_Bairro_SUM.shape)

### Total number of activities per bairro
Group_Ativ_Bairro = Ativ_Bairro.groupby(["CH_MUNICIPIO_BAIRRO"],as_index=False)["DS_ATIVIDADE"].value_counts()
Group_Ativ_Bairro = Group_Ativ_Bairro.groupby(["CH_MUNICIPIO_BAIRRO"],as_index=False)["count"].agg('sum')
csv_path = os.path.join(DATA_PATH_OUTPUT, "09ativeco_ESGV-Columns-Total-Atividade_por_BairroGV.csv")
Group_Ativ_Bairro.to_csv(csv_path, index=True, header=True)#, index_label = ["NO_MUNICIPIO","NO_BAIRRO"]
print("09", Group_Ativ_Bairro.shape)

### merge of number of different activities per bairro and total number of activities per bairro
Conta_Atividade_Bairro = pd.merge(Group_Ativ_Bairro_SUM,Group_Ativ_Bairro[["CH_MUNICIPIO_BAIRRO","count"]])

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "10ativeco_ESGV-Columns-Atividade_Bairro-Total.csv") #Somatorio_atividades_aposmerge.csv
Conta_Atividade_Bairro.to_csv(csv_path, index=True, header=True)
print("10", Conta_Atividade_Bairro.shape)


07 (269113, 5)
08 (1522, 2)
09 (1522, 2)
10 (1522, 3)


### Tabela de localidades (MUNICIPIOBAIRRO) IBGE

In [26]:
print("201", IBGE_Bairros.shape)
print(IBGE_Bairros.info())

201 (406, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Cód.                 406 non-null    object
 1   Município            406 non-null    object
 2   Bairro               406 non-null    object
 3   CH_MUNICIPIO_BAIRRO  406 non-null    object
dtypes: object(4)
memory usage: 12.8+ KB
None


## Cria (fuzzymerge) tabela com agrupamentos de localidade (MUNICIPIOBAIRRO)
- função fuzzymerge() não funciona com colunas de mesmo nome
- nova tabela ["Cód.","Município","Bairro","Num_Atividades","Total_Atividades"]
    - Num_Atividades = DS_ATIVIDADE = quantidade de atividades diferentes na localidade
    - Total_Atividades = count = quantidade de atividades na localidade

In [27]:
from levenpandas import fuzzymerge
import pandas as pd

### function fuzzymerge does not work well with key columns with the same name
# copy original dataframes with diferent key names
df1 = Conta_Atividade_Bairro.loc[:,Conta_Atividade_Bairro.columns]
df1['journal1'] = df1['CH_MUNICIPIO_BAIRRO']
df2 = IBGE_Bairros.loc[:,IBGE_Bairros.columns]
df2['journal2'] = df2['CH_MUNICIPIO_BAIRRO']
print(df2.info())

merged = fuzzymerge(df1, df2, left_on='journal1', right_on='journal2', threshold = 1, multi=-1)
merged = merged.dropna()

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "301Merged_CH_MUNICIPIO_BAIRRO.csv")
merged.to_csv(csv_path, index=True, header=True)
print("301", merged.shape)

### Creates dataframe with information from ativeco_ES (key column) matched with IBGE (key column)
columns_titles = ["Cód.","Município","Bairro","DS_ATIVIDADE","count","CH_MUNICIPIO_BAIRRO_x","CH_MUNICIPIO_BAIRRO_y" ]
Bairros = merged.loc[:,columns_titles]
Bairros = Bairros.rename(columns={'DS_ATIVIDADE': 'Num_Atividades',"count": "Total_Atividade"})
Bairros = Bairros.rename(columns={'CH_MUNICIPIO_BAIRRO_x': 'CH_MUNICIPIO_BAIRRO_ativeco_ES',"CH_MUNICIPIO_BAIRRO_y": "CH_MUNICIPIO_BAIRRO_IBGE"})
Bairros = Bairros.reset_index(drop=True)

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "302Bairros.csv")
Bairros.to_csv(csv_path, index=True, header=True)
print("302", Bairros.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Cód.                 406 non-null    object
 1   Município            406 non-null    object
 2   Bairro               406 non-null    object
 3   CH_MUNICIPIO_BAIRRO  406 non-null    object
 4   journal2             406 non-null    object
dtypes: object(5)
memory usage: 16.0+ KB
None
301 (386, 9)
302 (386, 7)


# MAPEAMENTO ATIVIDADES

### Cria tabela (de ativeco_ES) com agrupamentos de localidade (MUNICIPIOBAIRRO)
- usar coluna CH_MUNICIPIO_BAIRRO
- tabela [CH_MUNICIPIO_BAIRRO,DS_ATIVIDADE,count]
    - DS_ATIVIDADE = quantidade de atividades diferentes na localidade
    - count = quantidade de atividades na localidade

In [28]:
from levenpandas import fuzzymerge
import pandas as pd

### Assignment by copy and not by reference, keeping only the columns of interest
Ativ_Bairro = ativeco_ESGV.loc[:,["NO_MUNICIPIO","NO_BAIRRO","DS_ATIVIDADE","CH_ATIVIDADE","CH_MUNICIPIO_BAIRRO"]]

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "07ativeco_ESGV-Columns.csv") #Somatorio_atividades_aposmerge.csv
Ativ_Bairro.to_csv(csv_path, index=True, header=True)
print("07", Ativ_Bairro.shape)



### function fuzzymerge does not work well with key columns with the same name
# copy original dataframes with diferent key names
df1 = Ativ_Bairro.loc[:,Ativ_Bairro.columns]
df1 = df1.groupby(["CH_ATIVIDADE"],as_index=False)["CH_ATIVIDADE"].value_counts()
df1['journal1'] = df1['CH_ATIVIDADE']
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "400df1_atividades.csv") #Somatorio_atividades.csv
df1.to_csv(csv_path, index=True, header=True)
print("400", df1.shape)

df2 = CNAE23.loc[:,CNAE23.columns]
### remove Classe and Subclasse that may have the same name (duplicate) - see original CNAE23
df2 = df2.drop_duplicates(subset='Denominacao', keep='first')
df2['journal2'] = df2['CH_ATIVIDADE']
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "401df2_CNAE.csv") #Somatorio_atividades.csv
df2.to_csv(csv_path, index=True, header=True)
print("401", df2.shape)

merged = fuzzymerge(df1, df2, left_on='journal1', right_on='journal2', threshold = 1, multi=-1)
merged = merged.dropna()

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "402Merged_DS_ATIVIDADE_Dropna.csv")
merged.to_csv(csv_path, index=True, header=True)
print("402", merged.shape)

### Creates dataframe with information from ativeco_ES (key column) matched with CNAE (key column)
columns_titles = ["Divisão","DivNome","Subclasse","Denominacao","count",'CH_ATIVIDADE_x','CH_ATIVIDADE_y']
Atividades = merged.loc[:,columns_titles]
Atividades = Atividades.rename(columns={"count": "Total_Atividade"})
Atividades = Atividades.rename(columns={'CH_ATIVIDADE_x': 'CH_ATIVIDADE_ativeco_ES',"CH_ATIVIDADE_y": "CH_ATIVIDADE_CNAE"})
Atividades = Atividades.reset_index(drop=True)


### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "403Atividades.csv")
Atividades.to_csv(csv_path, index=True, header=True)
print("403", Atividades.shape)

### Total Activities by Divisão
g = Atividades.groupby(['Divisão'],as_index=False)['Total_Atividade'].agg('sum')
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "404Atividades_DivSum.csv")
g.to_csv(csv_path, index=True, header=True)
print("404", g.shape)



07 (269113, 5)
400 (1033, 3)
401 (1839, 6)
402 (965, 9)
403 (965, 7)
404 (83, 2)


# Construção da Tabela Bairro x Atividades
- Na verdade MunicipioBairro x DivisãoAtividades

###  Mantendo as colunas de interesse para o merge

In [29]:
### Assignment by copy and not by reference, keeping only the columns of interest
Ativ_Bairro = ativeco_ESGV.loc[:,["NO_MUNICIPIO","NO_BAIRRO","DS_ATIVIDADE","CH_ATIVIDADE","CH_MUNICIPIO_BAIRRO"]]
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "07ativeco_ESGV-Columns.csv") #Somatorio_atividades_aposmerge.csv
Ativ_Bairro.to_csv(csv_path, index=True, header=True)
print("07", Ativ_Bairro.shape)

### keeping only the columns of interest
print("302", Bairros.shape)
columns_titles = ["Cód.","Município","Bairro","CH_MUNICIPIO_BAIRRO_ativeco_ES","CH_MUNICIPIO_BAIRRO_IBGE" ]
Bairros = Bairros.loc[:,columns_titles]
Bairros = Bairros.reset_index(drop=True)
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "500Bairros.csv")
Bairros.to_csv(csv_path, index=True, header=True)
print("500", Bairros.shape)


### keeping only the columns of interest
print("404", Atividades.shape)
columns_titles = ["Divisão","DivNome","Denominacao","CH_ATIVIDADE_ativeco_ES","CH_ATIVIDADE_CNAE" ]
Atividades = Atividades.loc[:,columns_titles]
Atividades = Atividades.reset_index(drop=True)
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "501Atividades.csv")
Atividades.to_csv(csv_path, index=True, header=True)
print("501", Atividades.shape)



07 (269113, 5)
302 (386, 7)
500 (386, 5)
404 (965, 7)
501 (965, 5)


###  Merge ativeco_ES com IBGE = merged_bairros

In [30]:
### Merging original ativeco_ES (Ativ_Bairro) with Bairros filtered (levenstein) with IBGE (Bairros)
Bairro_ativeco_ES_IBGE = pd.merge(Ativ_Bairro, Bairros, left_on = 'CH_MUNICIPIO_BAIRRO', right_on = 'CH_MUNICIPIO_BAIRRO_ativeco_ES')
Bairro_ativeco_ES_IBGE = Bairro_ativeco_ES_IBGE.dropna()

### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "502Bairro_ativeco_ES_IBGE.csv")
Bairro_ativeco_ES_IBGE.to_csv(csv_path, index=True, header=True)
print("502", Bairro_ativeco_ES_IBGE.shape)

### Total info by bairro code
### if column NO_BAIRRO > 1, means that levenstein mapped two diferent strings to the same.
### for example, "ITACIBA" = "   ITACIBA" = "ITACIBÁ" = Cód. = 3201308046
#g = Bairro_ativeco_ES_IBGE.groupby(['Cód.'],as_index=False).nunique()
### Send to csv file
#csv_path = os.path.join(DATA_PATH_OUTPUT, "503Bairro_ativeco_ES_IBGE_nunique.csv")
#g.to_csv(csv_path, index=True, header=True)
#print("503", g.shape)


502 (242013, 10)


###  Merge ativeco_ES com CNAE = merged_atividades

In [31]:
### Merging original ativeco_ES (Ativ_Bairro) with Atividades filtered (levenstein) with CNAE (Atividades)
Atividades_ativeco_ES_CNAE = pd.merge(Ativ_Bairro, Atividades, left_on = 'CH_ATIVIDADE', right_on = 'CH_ATIVIDADE_ativeco_ES')
Atividades_ativeco_ES_CNAE = Atividades_ativeco_ES_CNAE.dropna()
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "504Atividades_ativeco_ES_CNAE.csv")
Atividades_ativeco_ES_CNAE.to_csv(csv_path, index=True, header=True)
print("504", Atividades_ativeco_ES_CNAE.shape)


### Total info by Divisão code
### if NO_BAIROO < CH_MUNICIPIO_BAIRRO, means that some municipalities have bairros with the same name.
### For example, "centro"
#g = Atividades_ativeco_ES_CNAE.groupby(['Divisão'],as_index=False).nunique()
### Send to csv file
#csv_path = os.path.join(DATA_PATH_OUTPUT, "505Atividades_ativeco_ES_CNAE_nunique.csv")
#g.to_csv(csv_path, index=True, header=True)
#print("505", g.shape)

504 (260879, 10)


###  Merge merged_bairros com merged_atividades =   merged_bairros_atividades

In [32]:
### Merging merged ativeco_ES,IBGE with merged ativeco_ES,CNAE

### keeping only the columns of interest from merged ativeco_ES,IBGE
## Assignment by copy and not by reference
columns_titles = ["Cód.","Município","Bairro","CH_MUNICIPIO_BAIRRO_ativeco_ES","CH_MUNICIPIO_BAIRRO_IBGE","CH_ATIVIDADE"]
Bairro_ativeco_ES_IBGE = Bairro_ativeco_ES_IBGE.loc[:,columns_titles]
Bairro_ativeco_ES_IBGE = Bairro_ativeco_ES_IBGE.reset_index(drop=True)
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "506Bairro_ativeco_ES_IBGE_Columns.csv")
Bairro_ativeco_ES_IBGE.to_csv(csv_path, index=True, header=True)
print("506", Bairro_ativeco_ES_IBGE.shape)

### Merging merged ativeco_ES CNAE with merged ativeco_ES,IBGE (Final dataframe)
AtividadesBairro_ativeco_ES_CNAE_IBGE = pd.merge(Bairro_ativeco_ES_IBGE, Atividades, left_on = 'CH_ATIVIDADE', right_on = 'CH_ATIVIDADE_ativeco_ES')
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "507AtividadesBairro_ativeco_ES_CNAE_IBGE.csv")
AtividadesBairro_ativeco_ES_CNAE_IBGE.to_csv(csv_path, index=True, header=True)
print("507", AtividadesBairro_ativeco_ES_CNAE_IBGE.shape)


### Assignment by copy and not by reference, keeping only the columns of interest from Final dataframe
columns_titles = ["Cód.","Município","Bairro","CH_MUNICIPIO_BAIRRO_IBGE","Divisão","DivNome","CH_ATIVIDADE_CNAE"]
AtividadesBairro_ativeco_ES_CNAE_IBGE = AtividadesBairro_ativeco_ES_CNAE_IBGE.loc[:,columns_titles]
#AtividadesBairro_ativeco_ES_CNAE_IBGE = AtividadesBairro_ativeco_ES_CNAE_IBGE.sort_values(["Divisão","Cód."])
AtividadesBairro_ativeco_ES_CNAE_IBGE = AtividadesBairro_ativeco_ES_CNAE_IBGE.reset_index(drop=True)
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "508AtividadesBairro_ativeco_ES_CNAE_IBGE_Columns.csv")
AtividadesBairro_ativeco_ES_CNAE_IBGE.to_csv(csv_path, index=True, header=True)
print("508", AtividadesBairro_ativeco_ES_CNAE_IBGE.shape)

### Total info by Divisão code
### Lost one division in above merges, from 83 to 82
g = AtividadesBairro_ativeco_ES_CNAE_IBGE.groupby(['Divisão'],as_index=False).nunique()
### Send to csv file
csv_path = os.path.join(DATA_PATH_OUTPUT, "509AtividadesBairros_ativeco_ES_CNAE_IBGE_Divisao_nunique.csv")
g.to_csv(csv_path, index=True, header=True)
print("509", g.shape)

506 (242013, 6)
507 (234553, 11)
508 (234553, 7)
509 (82, 7)


###  conta a quantidade de atividades por bairro em merged_bairros_atividades

In [33]:
### Sum of each activity in a neighborhood of a municipality
#AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts = AtividadesBairro_ativeco_ES_CNAE_IBGE.groupby(["Município","Bairro"],as_index=False)["DivNome"].value_counts()
AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts = AtividadesBairro_ativeco_ES_CNAE_IBGE.groupby(["Cód.","Município","Bairro"],as_index=False)[["DivNome","Divisão"]].value_counts()
csv_path = os.path.join(DATA_PATH_OUTPUT, "512AtividadesBairros_ativeco_ES_CNAE_IBGE_valuecounts.csv")
AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts.to_csv(csv_path, index=True, header=True)
print("512", AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts.shape)


512 (13667, 6)


### Constrói tabela final (linhas de atividades) x (colunas de atividades)

O conteúdo de cada célula da tabela contém a quantidade daquela atividade (coluna) naquele bairro (linha)

In [34]:
### Final Table Neighborhood x Activities
#AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot = AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts.pivot(index = ["Município","Bairro"] , columns="DivNome", values="count")
AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot = AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts.pivot(index = ["Cód.","Município","Bairro"] , columns=["DivNome","Divisão"], values="count")
AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot = AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot.replace(np.nan,0)
csv_path = os.path.join(DATA_PATH_OUTPUT, "513AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot.csv")
AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot.to_csv(csv_path, index=True, header=True)
print("513", AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot.shape)

513 (386, 82)


### Constrói saida
- arquivo com tabela final (linhas de atividades) x (colunas de atividades)

In [35]:
#Transformar estrutura pivot multindex em estrutura tradicional (flat)
Pivot_Flat = AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot.loc[:]
print(AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot.columns.values[3][1])
Pivot_Flat.columns   = ['_'.join(col) for col in AtividadesBairro_ativeco_ES_CNAE_IBGE_valuecounts_pivot.columns.values]

# sort Dataframe columns
Pivot_Flat = Pivot_Flat.sort_values(by = ["Município","Bairro"])

csv_path = os.path.join(DATA_PATH_OUTPUT, "600Pivot_Flat.csv")
Pivot_Flat.to_csv(csv_path, index=True, header=True)
print("600", Pivot_Flat.shape)
#Pivot_Flat.describe()

43
600 (386, 82)
